# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [62]:
import pandas as pd
import requests
import json
from azureml.core import Workspace, Datastore, Dataset, Experiment, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
# Read and clean the data

raw_data = pd.read_csv('data-unbalanced.csv')
raw_data = raw_data.dropna()
raw_data = pd.get_dummies(raw_data, columns=['PROD_CDE','PREP ASSOCIATE', 'FURN'])
print(raw_data.shape)

(26038, 93)


In [10]:
# Create a new Dataset from the raw_data
ws = Workspace.from_config()
datastore = Datastore.get(ws, 'workspaceblobstore')

dataset = Dataset.Tabular.register_pandas_dataframe(raw_data, datastore ,"Sinter", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/b22ba56f-425b-4554-95de-18018f960523/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [11]:
# Register the dataset
dataset = dataset.register(workspace = ws, name='Sinter_dataset', description='Sinter scrap dataset')

In [12]:
# choose a name for experiment
experiment_name = 'Sinter_AML'

experiment=Experiment(ws, experiment_name)

# Create a Copute target

In [13]:
cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HAS SCRAP AT SINTER",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Sinter_AML,AutoML_9e50608f-dce9-4706-aa7a-2480c162c927,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [27]:
best_run, best_model = remote_run.get_output()

best_model_metrics = best_run.get_metrics()


In [35]:
# Display the metrics of the best model

metrics = pd.DataFrame.from_dict(best_model_metrics, orient='index', columns=['metric value'])
metrics

,metric value
weighted_accuracy,0.99
AUC_micro,0.95
recall_score_micro,0.91
log_loss,0.31
precision_score_macro,0.71
norm_macro_recall,0.01
precision_score_weighted,0.87
accuracy,0.91
average_precision_score_micro,0.94
recall_score_weighted,0.91


In [58]:
#TODO: Save the best model
model_name = best_run.properties['model_name']

aml_model = remote_run.register_model(model_name=model_name, description='Best model form AutoML', tags=None)

print(remote_run.model_id)

AutoML9e50608fd24


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [59]:
# Define inference config

env = best_run.get_environment()
dummy_inference_config = InferenceConfig(
    environment= env,
    entry_script="./source_dir/echo_score.py"
)


# Define a deployment configuration

deploy_config = LocalWebservice.deploy_configuration(port=9000)

In [61]:
# Deploy the model

service = Model.deploy(
    ws,
    "myservice",
    [aml_model],
    dummy_inference_config,
    deploy_config
)

service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_4d4668f442e87b2baa899f119bc648ab
 ---> f1f4c52d7f98
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 1eefedddbf30
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImEwYTc2YmFkLTExYTEtNGEyZC05ODg3LTk3YTI5MTIyYzhlZCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE2MTIzNyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE2MTIzNyIsIndvcmtzcGFjZUlkIjoiZTZlOTNjMjgtMTczYi00YzlmLWE3ODQtNjNlMjJiZGQyMmI0In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 1e1a8a2c6c8f
 ---> 33b20edbb2b5
Step 4/5 : RUN mv '/var/azureml-app/tmpev5f8bpc.py' /var/azureml-app/main.py
 ---> Running in 2fee8fb9f39a
 ---> 74be44822f75
Step 5/5 : 

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [68]:
uri = service.scoring_uri
requests.get("http://localhost:9000")
headers = {"Content-Type":"application/json"}

data = {
    "query": "what color is the fox",
    "context": "the quick..."
}

data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)

print(response.json())

test is {'query': 'what color is the fox', 'context': 'the quick...'}


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
